<a href="https://colab.research.google.com/github/aljeshishe/colab/blob/main/dividends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/mbk-dev/okama/blob/master/examples/01%20howto.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

You can run the code examples in [Google Colab](https://colab.research.google.com/github/mbk-dev/okama/blob/master/examples/01%20howto.ipynb).

**To install the package:**

In [1]:
!pip install -U okama plotly

     |████████████████████████████████| 81 kB 3.4 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
import okama as ok
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12.0, 6.0]
pd.options.plotting.backend = 'plotly'
pd.options.display.max_rows = None

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [6]:
from gspread_dataframe import set_with_dataframe, get_as_dataframe

name = 'dividends'
# name = 'test'
sheet = gc.open(name).sheet1
new_df = get_as_dataframe(sheet)
tickers = new_df.dropna(how='all')[['symbol']]
tickers.head(5)

,symbol
0,LNZLP.MOEX
1,LNZL.MOEX
2,NLMK.MOEX
3,MTLRP.MOEX
4,CHMF.MOEX


In [ ]:
def get_div(row):
  x = ok.Asset(row.symbol)
  div = x.dividends
  div.index = div.index.to_timestamp()

  div_value = div[:'2022-01'].last('12M').sum() if '2021' in div.index else float('nan')
  forecast_div_value = div.last('12M').sum() if '2022' in div.index else float('nan')

  row['ticker'] = row.symbol.split('.')[0]
  row['div_value'] = div_value
  row['forecast_div_value'] = forecast_div_value
  row['inc'] = div_value / x.price * 100
  row['forecast_inc'] = forecast_div_value / x.price * 100
  row['name'] = x.name
  print(row.to_dict())
  return row

new_data_df = tickers.apply(get_div, axis='columns', result_type='expand').round(2)
new_data_df.head(5)

{'symbol': 'LNZLP.MOEX', 'ticker': 'LNZLP', 'div_value': 3699.27, 'forecast_div_value': nan, 'inc': 189.70615384615382, 'forecast_inc': nan, 'name': 'Lena Gold-Mining Public Joint Stock Company Lenzoloto'}
{'symbol': 'LNZL.MOEX', 'ticker': 'LNZL', 'div_value': 15219.5, 'forecast_div_value': nan, 'inc': 145.78065134099617, 'forecast_inc': nan, 'name': 'Lena Gold-Mining Public Joint Stock Company Lenzoloto'}


In [ ]:
def get_prices(row):
  x = ok.Asset(row.symbol)
  prices = x.close_daily
  prices.index = prices.index.to_timestamp()
  row['price'] = x.price
  row['30d_min'] = prices.last('30D').min()
  row['30d_max'] = prices.last('30D').max()
  row['30d_drowdown'] = x.price / row['30d_max'] * 100 - 100
  row['365d_min'] = prices.last('365D').min()
  row['365d_max'] = prices.last('365D').max()
  row['365d_drowdown'] = x.price / row['365d_max'] * 100 - 100
  print(row.to_dict())
  return row

new_data_df = new_data_df.apply(get_prices, axis='columns', result_type='expand').round(2)
new_data_df.head(5)


In [ ]:

set_with_dataframe(sheet, new_data_df, include_index=False)